In [1]:
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
np.set_printoptions(precision=3, suppress=True)

In [6]:
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


In [7]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [8]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
        file_path,
        batch_size=5,
        label_name=LABEL_COLUMN,
        na_value="?",
        num_epochs=1,
        ignore_errors=True,
        **kwargs
    )
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [9]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key, value.numpy()))

In [10]:
show_batch(raw_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'male']
age                 : [36. 23. 26. 23. 28.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 26.388 113.275   7.775   7.925  39.6  ]
class               : [b'First' b'First' b'Third' b'Third' b'First']
deck                : [b'E' b'D' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [11]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [51. 65. 28. 28. 29.]
n_siblings_spouses  : [0 0 0 0 1]
class               : [b'First' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'E' b'unknown' b'unknown' b'unknown']
alone               : [b'n' b'y' b'y' b'y' b'n']


In [16]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [34. 30. 50. 22.  4.]
n_siblings_spouses  : [0. 0. 1. 0. 0.]
parch               : [0. 0. 0. 0. 1.]
fare                : [ 26.55  106.425 106.425  10.517  13.417]


In [17]:
example_batch, labels_batch = next(iter(temp_dataset))

In [20]:
print(example_batch, labels_batch, sep = '\n')

OrderedDict([('age', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([28., 28.,  9., 24., 51.], dtype=float32)>), ('n_siblings_spouses', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 5., 0., 0.], dtype=float32)>), ('parch', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 0., 2., 0., 0.], dtype=float32)>), ('fare', <tf.Tensor: shape=(5,), dtype=float32, numpy=array([ 8.05 , 24.15 , 46.9  , 79.2  , 12.525], dtype=float32)>)])
tf.Tensor([0 1 0 0 0], shape=(5,), dtype=int32)


In [21]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [22]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy(), labels.numpy(), sep='\n\n')

[[45.     0.     1.    14.454]
 [35.     1.     1.    20.25 ]
 [28.     0.     2.    15.246]
 [51.     0.     0.    26.55 ]
 [38.     0.     0.    80.   ]]

[0 1 0 1 1]


In [23]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
        
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [25]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [26]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
class               : [b'Third' b'Second' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'n']
numeric             : [[25.     0.     0.     7.896]
 [39.     0.     0.    13.   ]
 [28.     0.     0.     7.25 ]
 [29.     1.     0.    21.   ]
 [37.     2.     0.     7.925]]


In [27]:
!pip install -q pandas

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [28]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [29]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [33]:
MEAN, STD

(array([29.631,  0.545,  0.38 , 34.385]),
 array([12.512,  1.151,  0.793, 54.598]))

In [34]:
def normalize_numeric_data(data, mean, std):
    return (data-mean)/std

In [35]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7efdde8f4d90>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [38]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [39]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[47.   ,  0.   ,  0.   , 34.021],
       [53.   ,  2.   ,  0.   , 51.479],
       [28.   ,  0.   ,  0.   ,  7.896],
       [34.   ,  0.   ,  0.   , 13.   ],
       [28.   ,  0.   ,  0.   ,  7.225]], dtype=float32)>

In [40]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.388, -0.474, -0.479, -0.007],
       [ 1.868,  1.264, -0.479,  0.313],
       [-0.13 , -0.474, -0.479, -0.485],
       [ 0.349, -0.474, -0.479, -0.392],
       [-0.13 , -0.474, -0.479, -0.497]], dtype=float32)

In [43]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [44]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
                key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [45]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [46]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


In [50]:
for column in categorical_columns + numeric_columns:
    print(column)

IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, norm

In [51]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [52]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     1.     0.     0.     0.     0.     0.     1.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     1.388 -0.474
 -0.479 -0.007  1.     0.   ]


In [54]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [55]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [56]:
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 0s 3ms/step - loss: 0.4905 - accuracy: 0.7464
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4181 - accuracy: 0.8198
Epoch 3/20
126/126 [==============================] - 0s 2ms/step - loss: 0.4083 - accuracy: 0.8150
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3838 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3809 - accuracy: 0.8214
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3790 - accuracy: 0.8325
Epoch 7/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3639 - accuracy: 0.8421
Epoch 8/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3609 - accuracy: 0.8421
Epoch 9/20
126/126 [==============================] - 0s 2ms/step - loss: 0.3484 - accurac

In [57]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 2ms/step - loss: 0.4427 - accuracy: 0.8447


Test Loss 0.4427066743373871, Test Accuracy 0.8446969985961914


In [62]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    prediction = tf.sigmoid(prediction).numpy()
    print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Predicted survival: 82.92%  | Actual outcome:  SURVIVED
Predicted survival: 30.26%  | Actual outcome:  DIED
Predicted survival: 3.81%  | Actual outcome:  SURVIVED
Predicted survival: 82.14%  | Actual outcome:  DIED
Predicted survival: 6.11%  | Actual outcome:  DIED
